In [1]:
import cx_Oracle
import os

In [2]:
os.environ["NLS_LANG"] = "POLISH_POLAND.AL32UTF8"
connection = cx_Oracle.connect("C##PPAWLUK", "#Lex2018", "144.21.89.60:1521/ORCL.591263988.oraclecloud.internal")

In [3]:
cur = connection.cursor()

In [4]:
cur.execute('DELETE FROM "C##PPAWLUK"."TEMAT_DOKUMENTU"')
cur.execute('commit')

In [5]:
dokumentsName = cur.execute('select distinct nazwa_dokumentu from dokument')

In [6]:
names = []

In [7]:
for dokument in dokumentsName:
    names.append(dokument[0])

In [8]:
dokumentsText = []

In [9]:
j = 0
for name in names:
    dokumentsText.append({'name': name, 'text': ''})
    text = cur.execute('select tekst from dokument where nazwa_dokumentu = \'{}\''.format(name))
    for t in text:
        dokumentsText[j]['text'] = dokumentsText[j]['text'] + t[0]
    j = j + 1

In [10]:
allTekst = ''

In [11]:
for z in dokumentsText:
    allTekst = allTekst + z['text']

In [12]:
allTekst = allTekst.split(' ')
allTekst.remove('')
allTekst = list(set(allTekst))

In [13]:
matrix = []

In [14]:
for word in allTekst:
    for dokument in names:
        matrix.append({"name":dokument,"word": word,'frequency':0})

In [15]:
for dokument in dokumentsText:
    text = dokument['text']
    text = text.split(' ')
    text.remove('')
    for m in matrix:
        if(m['name'] == dokument['name']):
            m['frequency'] = text.count(m['word'])

In [16]:
def themaDoc(matrix,nameDoc):
    tmpMatrix = []
    sum = 0
    for m in matrix:
        if(m['name'] == nameDoc):
            sum = sum + m['frequency']
            if(m['frequency'] > 1):
                tmpMatrix.append(m)
    sortMatrix = sorted(tmpMatrix, key=lambda x: x['frequency'],reverse=True)
    size = 10
    if( len(sortMatrix) < size):
        size = len(sortMatrix)
    tematy = ''
    for i in range(0,size,1):
        tematy = tematy + ' ' + sortMatrix[i]['word'] + ' ' + '{:.2f}%'.format((float(sortMatrix[i]['frequency'])/float(sum))*100)+','
        print("{} {:.2f}%".format(sortMatrix[i] ,(float(sortMatrix[i]['frequency'])/float(sum))*100))
    cur.execute('INSERT INTO "C##PPAWLUK"."TEMAT_DOKUMENTU" (NAZWA_DOKUMENTU, TEMATY) VALUES( \'{}\', \'{}\')'.format(nameDoc,tematy))
    return sortMatrix

In [17]:
tmpM = []
for name in names:
    tmpM.append(themaDoc(matrix,name))

{'name': 'D20181943.txt', 'word': 'sztandar', 'frequency': 17} 3.39%
{'name': 'D20181943.txt', 'word': 'centymetr', 'frequency': 15} 2.99%
{'name': 'D20181943.txt', 'word': 'załącznik', 'frequency': 14} 2.79%
{'name': 'D20181943.txt', 'word': 'rozporządzenie', 'frequency': 14} 2.79%
{'name': 'D20181943.txt', 'word': 'rok', 'frequency': 12} 2.40%
{'name': 'D20181943.txt', 'word': 'strona', 'frequency': 12} 2.40%
{'name': 'D20181943.txt', 'word': 'kolor', 'frequency': 11} 2.20%
{'name': 'D20181943.txt', 'word': 'płat', 'frequency': 11} 2.20%
{'name': 'D20181943.txt', 'word': 'pozycja', 'frequency': 10} 2.00%
{'name': 'D20181943.txt', 'word': 'numer', 'frequency': 10} 2.00%
{'name': 'D20181937.txt', 'word': 'artykuł', 'frequency': 15} 7.77%
{'name': 'D20181937.txt', 'word': 'postępowanie', 'frequency': 10} 5.18%
{'name': 'D20181937.txt', 'word': 'rok', 'frequency': 9} 4.66%
{'name': 'D20181937.txt', 'word': 'dzień', 'frequency': 8} 4.15%
{'name': 'D20181937.txt', 'word': 'ustawa', 'freque

In [18]:
for i in tmpM:
    print(i)

[{'name': 'D20181943.txt', 'word': 'sztandar', 'frequency': 17}, {'name': 'D20181943.txt', 'word': 'centymetr', 'frequency': 15}, {'name': 'D20181943.txt', 'word': 'załącznik', 'frequency': 14}, {'name': 'D20181943.txt', 'word': 'rozporządzenie', 'frequency': 14}, {'name': 'D20181943.txt', 'word': 'rok', 'frequency': 12}, {'name': 'D20181943.txt', 'word': 'strona', 'frequency': 12}, {'name': 'D20181943.txt', 'word': 'kolor', 'frequency': 11}, {'name': 'D20181943.txt', 'word': 'płat', 'frequency': 11}, {'name': 'D20181943.txt', 'word': 'pozycja', 'frequency': 10}, {'name': 'D20181943.txt', 'word': 'numer', 'frequency': 10}, {'name': 'D20181943.txt', 'word': 'sprawa', 'frequency': 10}, {'name': 'D20181943.txt', 'word': 'dziennik', 'frequency': 10}, {'name': 'D20181943.txt', 'word': 'państwowy', 'frequency': 9}, {'name': 'D20181943.txt', 'word': 'wewnętrzny', 'frequency': 7}, {'name': 'D20181943.txt', 'word': 'jednostka', 'frequency': 7}, {'name': 'D20181943.txt', 'word': 'dzień', 'freque

In [19]:
cur.execute('commit')

In [20]:
cur.close()
connection.close()